# Problem 1:
How many ways can one tile a $3 \times n$ wall using only $1\times 2$ and $2\times 1$ blocks?
$$

## Brute Force

In [8]:
# ChatGPT Generated
using BenchmarkTools

struct Wall
    grid::Matrix{Int}  # size 3×n
end

function all_walls_3xn(n::Int)
    # Create empty grid: 3 rows × n columns, 0 means "not filled yet"
    grid = fill(0, 3, n)
    solutions = Wall[]
    backtrack!(grid, 1, 1, 1, solutions)
    return solutions
end

function backtrack!(grid, r, c, next_id, solutions)
    n = size(grid, 2)

    # If c goes beyond n, move to next row:
    if c > n
        r += 1
        c = 1
    end

    # If r goes beyond 3, we've filled all rows => store a copy
    if r > 3
        push!(solutions, Wall(copy(grid)))
        return
    end

    # If current cell is already filled, just move on
    if grid[r, c] != 0
        backtrack!(grid, r, c+1, next_id, solutions)
        return
    end

    # 1) Try placing a vertical domino if there's space (r+1 within bounds)
    if r < 3 && grid[r+1, c] == 0
        grid[r,   c] = next_id
        grid[r+1, c] = next_id
        backtrack!(grid, r, c+1, next_id+1, solutions)
        # backtrack
        grid[r,   c] = 0
        grid[r+1, c] = 0
    end

    # 2) Try placing a horizontal domino if there's space (c+1 within bounds)
    if c < n && grid[r, c+1] == 0
        grid[r, c]   = next_id
        grid[r, c+1] = next_id
        backtrack!(grid, r, c+1, next_id+1, solutions)
        # backtrack
        grid[r, c]   = 0
        grid[r, c+1] = 0
    end
end

n = 24
walls = all_walls_3xn(n)
println("Number of ways to tile 3×$n with 2×1 bricks = ", length(walls))

Number of ways to tile 3×24 with 2×1 bricks = 5757961


## My solution $O(n^3)$

In [9]:
# Global cache for computed tiling configurations
tile_cache = Dict{Int, Vector{Int}}()

function tile(n::Int)
    if haskey(tile_cache, n)
        return tile_cache[n]
    end

    result = if n % 2 != 0
        Int[0]
    elseif n <= 2
        Int[3]
    else
        ways = zeros(Int, n ÷ 2)
        ways[1] = 2
        for i in 2:2:(n - 1)
            left, right = tile(i), tile(n - i)
            for (j, left_val) in enumerate(left)
                ways[j + 1] += left_val * right[1]
            end
        end
        ways
    end

    tile_cache[n] = result
    return result
end

# Example usage:
n = 24
val = @btime tile(n)
println("# Ways of tiling divided by # components for $n: ", val)
println("# Total Ways of tiling for $n: ", sum(val))


LoadError: InterruptException:

# Other solution $O(n^3)$

In [ ]:
using OffsetArrays

function ego(n)
    # init
    w = OffsetArray(zeros(Int, n, n), 1:n, 0:n-1)

    # base case
    w[2:2:n, 0] .= 2
    w[2,0] = 3

    # other cases
    for i in 2:n, j in 1:i-1
        w[i, j] = sum(w[k, j-1] * w[i-k, 0] for k in j:i-1)
    end

    # total number of configurations
    tot = sum( w[n,j] for j in 0:n-1 )

    # robustness index
    k = sum(i * w[n,i] for i in 0:n-1) / tot + 1

    return tot, k

end

ego(24)


(5757961, 9.708118550994007)

# $O(n^2)$ Solution

In [ ]:
# Possible but im too stupid for this

## Checking

In [ ]:
for n in 2:2:24
    if sum(tile(n)) != length(all_walls_3xn(n))
        println("$n failed")
    end
end

# Problem 2

# Problem 3
Given a sequential throw of $N$ dice $i = 1,\ldots,N$ with biases 
$p_1,\ldots,p_N : \{1,\ldots,6\} \to [0,1]$ 
(with $\sum_{x=1}^6 p_i(x) = 1$). 
Devise a polynomial time procedure to compute:
- What is the probability of having exactly $R$ times a repetition of the previous die?
- A "lucky run" is a subsequence of consecutive 6s in the throw. What is the most probable single throw with exactly $L$ lucky runs and a total of $S$ sixes?

In [53]:
# Helper function to decode an integer 'idx' into a base-6 sequence of length N,
# returning faces in 1..6
function decode_base6(idx::Int, N::Int)
    seq = Vector{Int}(undef, N)
    temp = idx
    for i in 1:N
        seq[i] = (temp % 6) + 1
        temp ÷= 6
    end
    return seq
end

# Compute the probability of a particular sequence `seq`
# given a probability matrix p of size N×6,
# where p[i, x] is the probability that the i-th throw shows face x (1-based).
function sequence_probability(seq::Vector{Int}, p::Matrix{Float64})
    prob = 1.0
    for i in 1:length(seq)
        prob *= p[i, seq[i]]
    end
    return prob
end

# Count the number of repeated pairs (i.e. consecutive equal faces)
function count_repeats(seq::Vector{Int})
    count = 0
    for i in 2:length(seq)
        if seq[i] == seq[i-1]
            count += 1
        end
    end
    return count
end

# Count the number of "lucky runs" of consecutive 6s
function count_lucky_runs(seq::Vector{Int})
    runs = 0
    in_run = false
    for face in seq
        if face == 6
            if !in_run
                runs += 1
                in_run = true
            end
        else
            in_run = false
        end
    end
    return runs
end

# Count the total number of sixes in the sequence
function count_sixes(seq::Vector{Int})
    return count(x -> x == 6, seq)
end

# Brute-force function:
# p: an N×6 matrix of probabilities (each row corresponds to one die throw)
# R: number of repeated pairs for which we accumulate probability,
# L: desired number of lucky runs (consecutive 6s) in the best sequence,
# S: desired total number of sixes in that best sequence.
function brute_force(p::Matrix{Float64}, R::Int, L::Int, S::Int)
    N = size(p, 1)          # number of dice throws (rows of p)
    prob_R = 0.0            # accumulates probability for sequences with exactly R repeats
    best_prob = 0.0         # best probability among sequences with L lucky runs & S sixes
    best_seq = Vector{Int}(undef, N)

    total_outcomes = 6^N
    for idx in 0:(total_outcomes - 1)
        seq = decode_base6(idx, N)
        prob = sequence_probability(seq, p)
        
        # Accumulate probability for sequences with exactly R repeated pairs
        if count_repeats(seq) == R
            prob_R += prob
        end
        
        # Track the best sequence with exactly L lucky runs and exactly S sixes
        if count_lucky_runs(seq) == L && count_sixes(seq) == S
            if prob > best_prob
                best_prob = prob
                best_seq = copy(seq)
            end
        end
    end
    return (prob_R, best_seq, best_prob)
end

# -------------------------
# Your provided probability matrix as a vector of vectors:
p_vec = [
    [1/2,   1/5,   1/5,   1/30, 1/30, 1/30],
    [1/6,   1/6,   1/6,   1/6,  1/6,  1/6 ],
    [1/8,   1/6,   1/8,   7/36, 7/36, 7/36],
    [1/3,   1/4,   1/18,  1/4,  1/18, 1/18],
    [1/5,   1/5,   1/5,   2/15, 2/15, 2/15],
    [1/4,   1/4,   1/4,   1/8,  1/16, 1/16],
]

# Convert vector of vectors to a Matrix.
# hcat(p_vec...) creates a matrix with each inner vector as a column,
# so we transpose it to have each as a row.
p = Float64.(hcat(p_vec...)')

# Example parameters:
# R = 1 repeated pair, L = 1 lucky run of consecutive 6s, S = 2 sixes total.
R = 2
L = 1
S = 1

prob_R, best_seq, best_prob = brute_force(p, R, L, S)
println("Total probability for exactly $R repeated pairs: ", prob_R)
println("Best sequence with $L lucky run(s) and $S sixes: ", best_seq)
println("Probability of that sequence: ", best_prob)


Total probability for exactly 2 repeated pairs: 0.16452874942844686
Best sequence with 1 lucky run(s) and 1 sixes: [1, 6, 4, 1, 1, 1]
Probability of that sequence: 0.0002700617283950617


## My Solution

In [ ]:
function U(R,p)
    for k in 1:6
        for i in 1:length(p)
            L
        end
    end
end

[0.06666666666666667, 0.03333333333333333, 0.041666666666666664, 0.16666666666666666]


U (generic function with 1 method)

In [ ]:
function table(p)
    
end



# for dice in p
#     println(sum(dice))
# end

T = table(p)

function U(i,R,d)
    
end

U (generic function with 1 method)